In [11]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
from fileUtils import getBasename, getDirname, getBaseFilename
from fsUtils import isFile, isDir, moveFile, removeFile, mkDir, setFile, moveFile
from searchUtils import findDirs, findExt, findNearest

################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed, getDateTime, isDate
from webUtils import getHTML, getWebData
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from time import sleep
from multiprocessing import Pool
from tqdm import tqdm

################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb

################################################################################
## Music Stuff
################################################################################
from myMusicDBMap import myMusicDBMap
from musicData import musicData
from musicBase import myMusicBase
from matchMyMusic import matchMyMusic
from matchMusicName import myMusicName
from mergeDB import searchForMutualDBEntries, searchForMutualArtistDBEntries

################################################################################
## Chart Stuff
################################################################################
from top40charts import top40, top40chart, top40starter
from fullCharts import fullCharts
from artistIgnores import getArtistIgnores
from matchChartMusic import matchChartMusic
from chartAnalysisClass import chartAnalysisClass

################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Python: {0}".format(sys.version))

from pandas import date_range, DataFrame
basedir = getcwd()
print("Basedir = {0}".format(basedir))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Basedir = /Users/tgadfort/Documents/code/charts
Notebook Last Run Initiated: 2020-10-29 22:51:56.615694


In [12]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()
mcm = matchChartMusic(mdb)

Counter({'Discogs': 4624, 'AllMusic': 4512, 'MusicBrainz': 4273, 'LastFM': 3415, 'CDandLP': 760, 'RockCorner': 551, 'DatPiff': 518, 'AceBootlegs': 176, 'RateYourMusic': 65, 'MusicStack': 4, 'MetalStorm': 0})
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 752449 ID -> Name entries
    Found 687084 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: False
    Found 54014 ID -> Name entries
    Found 50150 Name -> ID entries
    Found 229101 Albums
=========================   musicbrainz   =========================
Summary Statistics For DB: MusicBrainz
    Using Known Artists: False
    Found 133914 ID -> Name entries
    Found 112971 Name -> ID entries
    Found 0 Albums
=========================   acebootlegs   =========================
Summary Statistics For DB: AceBootlegs
    Using

In [13]:
%load_ext autoreload
%autoreload

chartTypes = ["Top40", "Billboard"]
cacs = {}
from chartAnalysisClass import chartAnalysisClass
for chartType in chartTypes:
    cac = chartAnalysisClass(chartType, init=False, full=False)
    cac.setFull(mdb, mcm)
    cac.setChartArtists()
    cac.findArtistsMatchedStatus()
    cacs[chartType] = cac
    print("\n"*2)

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Adding 326 known multi delim artists.
Counter({'Discogs': 4624, 'AllMusic': 4512, 'MusicBrainz': 4273, 'LastFM': 3415, 'CDandLP': 760, 'RockCorner': 551, 'DatPiff': 518, 'AceBootlegs': 176, 'RateYourMusic': 65, 'MusicStack': 4, 'MetalStorm': 0})
Loading data from masterRename.yaml
  --> This file is 31.3kB.
Loading masterRename.yaml
-------------------------------------------------- Getting Artist Album Data --------------------------------------------------
	Loading Artist Albums Data From currentTop40ArtistAlbumData.p
Loading data from currentTop40ArtistAlbumData.p
  --> This file is 903.7kB.
Loading currentTop40ArtistAlbumData.p
-----------------------------------------------------------------------------------------------------------------------------



-------------------------------------------------- Getting Full Chart Artist Album Data --------------------------------------------------
	Loa

In [14]:
## Basic stuff
%load_ext autoreload
%autoreload
artistRenames = getFile("masterRename.yaml")
from musicData import musicData
from musicDBData import musicDBData
from musicArtistData import musicArtistData
md = musicData(init=True)


### Set MusicDB Data
mdbChartType = next(iter(cacs.keys()))
md.setMusicDBData(cacs[mdbChartType].mdb, debug=True)

### Set ChartDB Data
for chartType,chartData in cacs.items():
    md.setChartDBData(chartType, chartData.matchedChartResults, debug=True)
    
### Set Rename Data    
md.setRenameData(artistRenames, debug=True)

### Save
md.save()


### Master DF
md.createMasterDF()

_ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Counter({'Discogs': 4624, 'AllMusic': 4512, 'MusicBrainz': 4273, 'LastFM': 3415, 'CDandLP': 760, 'RockCorner': 551, 'DatPiff': 518, 'AceBootlegs': 176, 'RateYourMusic': 65, 'MusicStack': 4, 'MetalStorm': 0})
====> Will add 4763 entries for source Music
	After adding 4763 entries for source Music there are now 4763 artist entries
====> Will add 16408 entries for source Top40
	After adding 16408 entries for source Top40 there are now 19819 artist entries
====> Will add 5994 entries for source Billboard
	After adding 5994 entries for source Billboard there are now 22581 artist entries
====> Will add 845 entries for source Renames
	After adding 845 entries for source Renames there are now 22634 artist entries
There are 22634 known artists
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/musicData.p
  --> This file is 2.8MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/musi

In [15]:
df = getFile("/Users/tgadfort/opt/anaconda3/envs/py37/musicdb/masterMusicData.p")

In [20]:
df[~df["Top40"].isna()]

Music                               Billboard                                    Top40  Status
Artist         DB                                                                                                                                          
*NSYNC         AllMusic                                 0000516929                              0000516929                               0000516929       1
               Discogs                                       69433                                   69433                                    69433       1
               LastFM                                  67689072579                             67689072579                              67689072579       1
               MusicBrainz  59621135211322028167760795472347712673  59621135211322028167760795472347712673   59621135211322028167760795472347712673       1
10,000 Maniacs AllMusic                                 0000501081                              0000501081                               0000501081       1
...                                                            ...                                     ...                                      ...     ...
Dj Red 5       Discogs                                         NaN                                     NaN                                    10762       1
Gaudino        Discogs                                         NaN                                     NaN                                    45912       1
               MusicBrainz                                     NaN                                     NaN   18440450459113366671347778925912947906       1
Jack U         Discogs                                         NaN                                     NaN                                  4010464       1
Mako           MusicBrainz                                     NaN                                     NaN  306795885842547601457931621214439938247       1

[38905 rows x 4 columns]